In [5]:
import os
from pathlib import Path
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

# ====================================================
# CONFIGURATION
# ====================================================
# Base SENT folder path (update if needed)
sent_folder = Path("/home/the_fat_cat/Documents/GitHub/dysarthria-classification/data/dysarthria_raw/dysarthria_raw_original/NDDS/NDDS/SPEECH/SENT/")

# Output folder for extracted single-word .wav files
output_dir = Path("/home/the_fat_cat/Documents/GitHub/dysarthria-classification/data/dysarthria_raw/dysarthria_raw_single-words/NDDS-single-words/")
output_dir.mkdir(parents=True, exist_ok=True)

# Minimum valid word duration (in seconds) to consider it a full word
MIN_DURATION = 0.3  
# Amplitude threshold below which a segment is considered silent/noise
SILENCE_THRESHOLD = 0.005

# ====================================================
# HELPER FUNCTIONS
# ====================================================

def parse_seg_file(seg_path):
    """
    Parse a .SEG file to extract word-level boundaries.
    Each line is expected to be:
        <segment_label> <start_time_microsec> <end_time_microsec> [other info...]
    Only lines whose label starts with "W" (case-insensitive) are used.
    
    Returns:
        List of tuples: [(start_sec, end_sec), ...]
    """
    segments = []
    with seg_path.open("r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 3:
                continue
            if not parts[0].upper().startswith("W"):
                continue
            try:
                start_micro = float(parts[1])
                end_micro = float(parts[2])
            except ValueError:
                print(f"Error parsing times in {seg_path}: {line.strip()}")
                continue
            start_sec = start_micro / 1e6
            end_sec = end_micro / 1e6
            segments.append((start_sec, end_sec))
    return segments

def is_silent(audio_segment, sr, threshold=SILENCE_THRESHOLD, min_duration=MIN_DURATION):
    """
    Determine if an audio segment is effectively silent or too short.
    
    Args:
        audio_segment (np.array): Audio samples.
        sr (int): Sample rate.
        threshold (float): Mean absolute amplitude threshold below which the segment is silent.
        min_duration (float): Minimum duration (in seconds) required for a valid word.
    
    Returns:
        True if the segment is silent or too short, else False.
    """
    duration = len(audio_segment) / sr
    if duration < min_duration:
        return True
    mean_amp = np.mean(np.abs(audio_segment))
    return mean_amp < threshold

# ====================================================
# MAIN PROCESSING
# ====================================================

# Get all speaker folders in the SENT folder.
# These folders should be like "BB", "BK", "SC", etc.
speaker_folders = sorted([f for f in sent_folder.iterdir() if f.is_dir()], key=lambda x: x.name.upper())

# Assign speaker numbers (only for dysarthric speakers).
# Exclude control WAV files within folders based on their filename (starting with "JP")
speaker_map = {}
speaker_index = 1
for folder in speaker_folders:
    speaker_id = folder.name.upper()
    # We include the entire folder, but we will skip any WAV file in the folder that starts with "JP"
    if speaker_id.startswith("JP"):
        print(f"Skipping control speaker folder: {speaker_id}")
        continue
    speaker_map[speaker_id] = f"{speaker_index:02d}"
    speaker_index += 1

print("Speaker mapping (dysarthric only):")
for spk, num in speaker_map.items():
    print(f"{spk}: {num}")

# Process each dysarthric speaker folder
for speaker_folder in tqdm(speaker_folders, desc="Processing Speakers"):
    speaker_id = speaker_folder.name.upper()
    if speaker_id not in speaker_map:
        continue  # Skip control speakers

    speaker_num = speaker_map[speaker_id]
    
    # Define subfolders: WAV, TXT, SEG
    wav_dir = speaker_folder / "WAV"
    txt_dir = speaker_folder / "TXT"
    seg_dir = speaker_folder / "SEG"
    
    if not (wav_dir.exists() and txt_dir.exists() and seg_dir.exists()):
        print(f"Warning: Missing one of WAV, TXT, or SEG for speaker {speaker_id}. Skipping.")
        continue

    # Running counter for extracted words for this speaker
    word_counter = 0

    # Process each sentence (.wav file) in the WAV folder (case-insensitive)
    wav_files = sorted(wav_dir.glob("*.[Ww][Aa][Vv]"))
    for wav_file in tqdm(wav_files, desc=f"Processing {speaker_id}", leave=False):
        # Skip any control file (filename starting with "JP")
        if wav_file.stem.upper().startswith("JP"):
            continue
        
        base_name = wav_file.stem  # e.g., "BB1"
        
        # Construct the corresponding .TXT and .SEG file paths
        txt_file = txt_dir / f"{base_name}.TXT"
        seg_file = seg_dir / f"{base_name}.SEG"
        
        if not txt_file.exists() or not seg_file.exists():
            print(f"Warning: Missing TXT or SEG for {wav_file.name}. Skipping sentence.")
            continue
        
        # Read transcript (assume a single line)
        try:
            transcript_line = txt_file.read_text().strip()
        except Exception as e:
            print(f"Warning: Could not read {txt_file.name}: {e}. Skipping sentence.")
            continue
        transcript_words = transcript_line.split()
        
        # Parse segmentation file for boundaries
        segments = parse_seg_file(seg_file)
        
        # If the number of segments is exactly one more than transcript words, trim the extra segment.
        if len(segments) == len(transcript_words) + 1:
            segments = segments[:len(transcript_words)]
            print(f"Info: Trimmed extra segment for {wav_file.name}.")
        elif len(transcript_words) != len(segments):
            print(f"Warning: Mismatch in word count for {wav_file.name}: {len(transcript_words)} words vs. {len(segments)} segments. Skipping sentence.")
            continue

        # Load the full audio file (preserving original sample rate)
        try:
            audio, sr = librosa.load(wav_file, sr=None)
        except Exception as e:
            print(f"Warning: Error loading {wav_file.name}: {e}. Skipping sentence.")
            continue

        # Extract each word segment
        for i, ((start_sec, end_sec), word_text) in enumerate(zip(segments, transcript_words), start=1):
            start_sample = int(start_sec * sr)
            end_sample = int(end_sec * sr)
            word_audio = audio[start_sample:end_sample]

            # Check for silence/short segment
            if is_silent(word_audio, sr):
                print(f"Info: Skipping silent/short segment in {wav_file.name} word {i}.")
                continue

            # Increment word counter and construct filename
            word_counter += 1
            out_filename = f"NDDS_m_{speaker_num}_afflicted_{word_counter:03d}_a.wav"
            out_path = output_dir / out_filename

            try:
                sf.write(out_path, word_audio, sr)
            except Exception as e:
                print(f"Warning: Error writing {out_filename}: {e}. Skipping this word.")
                continue

print("Word extraction complete.")
print(f"Check the output folder: {output_dir}")


Speaker mapping (dysarthric only):
BB: 01
BK: 02
BV: 03
FB: 04
JF: 05
KS: 06
LL: 07
MH: 08
RK: 09
RL: 10
SC: 11


Processing Speakers:   9%|▉         | 1/11 [00:00<00:01,  5.21it/s]

Info: Skipping silent/short segment in BB1.WAV word 1.
Info: Skipping silent/short segment in BB1.WAV word 3.
Info: Skipping silent/short segment in BB1.WAV word 5.
Info: Trimmed extra segment for BB10.WAV.
Info: Skipping silent/short segment in BB10.WAV word 1.
Info: Skipping silent/short segment in BB10.WAV word 2.
Info: Skipping silent/short segment in BB10.WAV word 4.
Info: Skipping silent/short segment in BB10.WAV word 6.
Info: Skipping silent/short segment in BB11.WAV word 1.
Info: Skipping silent/short segment in BB11.WAV word 3.
Info: Skipping silent/short segment in BB11.WAV word 4.
Info: Trimmed extra segment for BB12.WAV.
Info: Skipping silent/short segment in BB12.WAV word 1.
Info: Skipping silent/short segment in BB12.WAV word 2.
Info: Skipping silent/short segment in BB12.WAV word 3.
Info: Skipping silent/short segment in BB12.WAV word 4.
Info: Skipping silent/short segment in BB12.WAV word 6.
Info: Trimmed extra segment for BB13.WAV.
Info: Skipping silent/short segment i


Processing BK:   0%|          | 0/148 [00:00<?, ?it/s]

Info: Trimmed extra segment for BK10.WAV.
Info: Skipping silent/short segment in BK10.WAV word 1.
Info: Skipping silent/short segment in BK10.WAV word 4.
Info: Skipping silent/short segment in BK10.WAV word 6.


...

Info: Skipping silent/short segment in SC1.WAV word 1.
Info: Skipping silent/short segment in SC1.WAV word 5.
Info: Skipping silent/short segment in SC10.WAV word 1.
Info: Skipping silent/short segment in SC10.WAV word 5.
Info: Skipping silent/short segment in SC11.WAV word 1.
Info: Skipping silent/short segment in SC11.WAV word 5.
Info: Skipping silent/short segment in SC12.WAV word 1.
Info: Skipping silent/short segment in SC12.WAV word 5.
Info: Trimmed extra segment for SC13.WAV.
Info: Skipping silent/short segment in SC13.WAV word 1.
Info: Skipping silent/short segment in SC13.WAV word 2.
Info: Skipping silent/short segment in SC13.WAV word 6.
Info: Trimmed extra segment for SC14.WAV.
Info: Skipping silent/short segment in SC14.WAV word 1.
Info: Skipping silent/short segment in SC14.WAV word 2.
Info: Skipping silent/short segment in SC14.WAV word 6.
Info: Trimmed extra segment for SC15.WAV.
Info: Skipping silent/short segment in SC15.WAV word 1.
Info: Skipping silent/short segment 


Processing SC:  74%|███████▎  | 109/148 [00:00<00:00, 1064.80it/s]

Info: Skipping silent/short segment in SC22.WAV word 5.
Info: Skipping silent/short segment in SC23.WAV word 1.
Info: Skipping silent/short segment in SC23.WAV word 5.
Info: Trimmed extra segment for SC24.WAV.
Info: Skipping silent/short segment in SC24.WAV word 1.
Info: Skipping silent/short segment in SC24.WAV word 2.
Info: Skipping silent/short segment in SC24.WAV word 4.
Info: Skipping silent/short segment in SC24.WAV word 6.
Info: Skipping silent/short segment in SC25.WAV word 1.
Info: Skipping silent/short segment in SC25.WAV word 5.
Info: Trimmed extra segment for SC26.WAV.
Info: Skipping silent/short segment in SC26.WAV word 1.
Info: Skipping silent/short segment in SC26.WAV word 2.
Info: Skipping silent/short segment in SC26.WAV word 6.
Info: Skipping silent/short segment in SC27.WAV word 1.
Info: Skipping silent/short segment in SC27.WAV word 5.
Info: Trimmed extra segment for SC28.WAV.
Info: Skipping silent/short segment in SC28.WAV word 1.
Info: Skipping silent/short segmen


Processing Speakers: 100%|██████████| 11/11 [00:02<00:00,  4.15it/s]A

Info: Skipping silent/short segment in SC73.WAV word 1.
Info: Skipping silent/short segment in SC73.WAV word 5.
Info: Skipping silent/short segment in SC74.WAV word 1.
Info: Skipping silent/short segment in SC74.WAV word 5.
Info: Skipping silent/short segment in SC8.WAV word 1.
Info: Skipping silent/short segment in SC8.WAV word 5.
Info: Skipping silent/short segment in SC9.WAV word 1.
Info: Skipping silent/short segment in SC9.WAV word 3.
Info: Skipping silent/short segment in SC9.WAV word 5.
Word extraction complete.
Check the output folder: /home/the_fat_cat/Documents/GitHub/dysarthria-classification/data/processed/NDDS-single-words
